# Preprocesamiento de hilos en 4chan

En este cuaderno se procesan las publicaciones de 4chan, obtenidas a través de [web scrapping](./Scraper-4chan). Estas publicaciones pertenecen a múltiples [hilos que son considerados pertinentes]() para el objetivo de la investigación, ya sea por su idiosincrática nocividad en el uso del lenguaje, por la expresión misma de ideaciones/intenciones suicidas, o por contener sugerencias para llevar a cabo el suicidio.

El objetivo de este cuaderno es retirar caracteres inútiles, transformar el contenido de cada publicación a minúsculas y eliminar los saltos de lí­nea en cada publicación. Además, es posible emplear una lista de palabras objetivo y localizar las publicaciones que contengan alguna de ellas.

Para esta tarea se utilizará la librería de `pandas`, pues su parser de archivos `.csv` admite por defecto el uso de saltos de lí­nea en los campos de texto.

In [1]:
import fnmatch
import pandas
import os
import re

A raíz de que se han consultado (y se consultarán) múltiples hilos, este script lee todas las publicaciones de los hilos y realiza múltiples operaciones sobre ellos.

A su vez, se crea una lista de nombres de archivos ya procesados, para excluirlos de la lectura cuando se trabaje en sesiones posteriores.

In [2]:
# Función para encontrar archivos

def encontrar_archivos(patron: str, ruta_base: str, lista_ex: list, verbose: bool = False):

    archivos = []
    index = 1

    for root, _, files in os.walk(ruta_base):

        for name in files:

            if fnmatch.fnmatch(name, patron) and name not in lista_ex:
                archivos.append(os.path.join(root, name))
                if verbose:
                    print(f'Archivo #{index}: {name}')
                    index += 1

    return archivos

In [3]:
# Leemos los archivos ya procesados desde un archivo de texto
archivos_procesados = []

with open('./datos/lista_procesados.txt', 'r') as archivo: 
    for linea in archivo:
        archivos_procesados.append(linea.strip())

archivos_de_la_sesion = []
archivos_procesados[0:3]

['ahorcarme_complete.csv', 'colgarme_complete.csv', '']

In [4]:
rutas_archivos = []
rutas_base = [
    './datos/hilos_4chan',
]

for ruta in rutas_base:
    rutas_archivos.extend(
        encontrar_archivos(
            patron='*.csv',
            ruta_base=ruta,
            lista_ex=archivos_procesados,
            verbose=True
        )
    )

Archivo #1: 1436774 - comments & replies.csv
Archivo #2: 17913796 - comments & replies.csv
Archivo #3: 17918614 - comments & replies.csv
Archivo #4: 17927427 - comments & replies.csv
Archivo #5: 17927993 - comments & replies.csv
Archivo #6: 17928132 - comments & replies.csv
Archivo #7: 17929614 - comments & replies.csv
Archivo #8: 17929761 - comments & replies.csv
Archivo #9: 17930158 - comments & replies.csv
Archivo #10: 17930355 - comments & replies.csv
Archivo #11: 1973636 - comments & replies.csv
Archivo #12: 2458998 - comments & replies.csv
Archivo #13: 2478234 - comments & replies.csv
Archivo #14: 2481228 - comments & replies.csv
Archivo #15: 2481880 - comments & replies.csv
Archivo #16: 2483351 - comments & replies.csv
Archivo #17: 2484336 - comments & replies.csv
Archivo #18: 2486136 - comments & replies.csv
Archivo #19: 2486883 - comments & replies.csv
Archivo #20: 2487974 - comments & replies.csv
Archivo #21: 2669577 - comments & replies.csv
Archivo #22: 2694142 - comments & 

Los archivos contienen saltos de línea dentro de los campos, y las respuestas a un comentario raíz tienen el prefijo "(REPLY) >>", seguido del número del usuario. Esto se eliminará de las publicaciones.

In [5]:
def pre_procesar_dataset(dataset: pandas.DataFrame):

    # Se eliminan los saltos de línea y los nombres de usuario en las publicaciones/respuestas
    dataset['comment/reply'] = dataset['comment/reply'].str.replace('\n', ' ')
    dataset['comment/reply'] = dataset['comment/reply'].str.replace(
        '((\(REPLY\))?\s>>.{8})|>', '', regex=True)
    dataset['comment/reply'] = dataset['comment/reply'].str.lstrip()
    # Se retiran las columnas que no se usarán
    columnas = [
        'post_id',
        'subject',
        'name',
        'is_op?',
    ]
    dataset.drop(columns=columnas, inplace=True)
    # Se renombran las columnas -> Esta adecuación se hace para darles un formato similar a los dataframes de Twitter,
    # para que en algún momento sean más sencillos de procesar bajo el mismo conjunto de instrucciones
    dataset.rename(
        columns={
            'date_time': 'Date',
            'comment/reply': 'Content',
            'url': 'URL'
        },
        inplace=True
    )

    # Se eliminan las filas que no tienen contenido
    dataset.dropna(subset=['Content'], inplace=True)

def obtener_hilo_desde_ruta(ruta: str):
    # Para conocer el hilo, se obtiene el nombre del directorio padre de la ruta
    nombre_hilo = os.path.basename(os.path.dirname(ruta))
    # Se eliminan los números y el guión del nombre del hilo
    nombre_hilo = re.sub(r'[0-9]*\s-\s', '', nombre_hilo)

    return nombre_hilo

In [6]:
# Se leen los archivos .csv encontrados
datasets = []

for ruta in rutas_archivos:

    dataset = pandas.read_csv(ruta)
    pre_procesar_dataset(dataset)

    datasets.append(
        {
            "hilo": obtener_hilo_desde_ruta(ruta),
            "datos": dataset,
        }
    )

In [7]:
# Observemos un ejemplo
datasets[4]['datos'].head()

,Date,Content,URL
0,"Jun-23-2023, 09:17:41",My balls is now hurts so much. Cirno is truly ...,http://boards.4chan.org/bant/thread/17927993/m...
1,"Jun-23-2023, 09:18:26",The humor stems from the fact that it is porno...,http://boards.4chan.org/bant/thread/17927993/m...
2,"Jun-23-2023, 09:18:45",i almost thought for a moment that the spic po...,http://boards.4chan.org/bant/thread/17927993/m...
3,"Jun-23-2023, 09:57:52",wtf Mono,http://boards.4chan.org/bant/thread/17927993/m...
4,"Jun-23-2023, 12:35:21",you need to go fucking kill yourself tiny whit...,http://boards.4chan.org/bant/thread/17927993/m...


In [9]:
# Agregamos una columna con el nombre del hilo
for dataset in datasets:
    dataset['datos']['Thread'] = dataset['hilo']

Una vez preparados los datos, podemos guardar una copia de ellos para su futuro análisis.

In [19]:
# Creamos la ruta donde se guardarán los archivos
ruta_guardado = "./datos_limpios/4chan"
sufijo = "_cleaned.csv"

os.makedirs(ruta_guardado, exist_ok=True)
print(f'Ruta creada: {ruta_guardado}')

Ruta creada: ./datos_limpios/4chan


In [99]:
for dict_ in datasets:
    hilo = dict_['hilo']
    dataset = dict_['datos']
    nombre_guardado_full = f"{ruta_guardado}/{hilo}{sufijo}"
    dataset.to_csv(nombre_guardado_full, index=False)

Ahora que agregamos información del hilo al que pertenece, podemos juntar todas las publicaciones en un solo archivo.

In [14]:
dataset_full = pandas.concat([dict_['datos'] for dict_ in datasets])
# Guardamos el dataset sin números de fila
dataset_full.to_csv(f"{ruta_guardado}/dataset_4chan.csv", index=False)

Además, podemos leer una serie de palabras relevantes de un archivo, para después obtener únicamente publicaciones que las contienen.

In [12]:
ruta_archivo_palabras = './datos_p_filtrar/palabras_objetivo.txt'

with open(ruta_archivo_palabras, 'r') as archivo:
    palabras = archivo.readlines()
    palabras = [palabra.replace('\n', '') for palabra in palabras]
    print("Algunas de las palabras son: ", palabras[0:5])

Algunas de las palabras son:  ['suicide', 'suicidal', 'suic', 'self-harm', 'self-injury']


In [15]:
# Tomamos el dataset completo y recuperamos las publicaciones que contengan las palabras objetivo
dataset_full = pandas.read_csv(f"{ruta_guardado}/dataset_4chan.csv")

# Quitamos las filas que no tienen contenido
dataset_full.dropna(subset=['Content'], inplace=True)

# Preprocesamos la lista de palabras objetivo para que sean útiles como expresiones regulares
palabras = [palabra.replace(' ', '\\s') for palabra in palabras]

# Mantenemos las filas que contengan las palabras objetivo
dataset_full = dataset_full[
    dataset_full['Content'].str.contains(
        '|'.join(palabras),
        regex=True
    )
]

In [16]:
dataset_full.to_csv(f"{ruta_guardado}/dataset_4chan_keywords_only.csv", index=False)

Para posteriores análisis, podemos decidir guardar los archivos procesados en la lista, para indicar que terminamos de correrlos por el pipeline.

In [17]:
# Anexamos los archivos procesados en esta sesión a la lista de archivos procesados

with open('./datos/lista_procesados.txt', 'a') as archivo:
    for ruta in rutas_archivos:
        hilo = obtener_hilo_desde_ruta(ruta)
        nombre_archivo = os.path.basename(ruta)
        archivo.write(f"{nombre_archivo}\n")